In [ ]:
from transformers import AutoModelForImageClassification
import torch

# Load the model
model = AutoModelForImageClassification.from_pretrained("amd/FDViT_ti", trust_remote_code=True)

# Set the model to evaluation mode
model.eval()

#
model.save_pretrained("model", save_format='tf')

# Extract the state dictionary (weights)
state_dict = model.state_dict()

# Save the weights to a file
torch.save(state_dict, "FDViT_ti_weights.pth")



config.json:   0%|          | 0.00/72.2k [00:00<?, ?B/s]

configuration_fdvit.py:   0%|          | 0.00/5.01k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/amd/FDViT_ti:
- configuration_fdvit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_fdvit.py:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/amd/FDViT_ti:
- modeling_fdvit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


###FDViT Architecture:

In [ ]:
print(model)

FDViTForImageClassification(
  (fdvit): FDViTModel(
    (encoder): FDViTEncoder(
      (patch_embed): FDViTEmbeddings(
        (conv): Conv2d(3, 64, kernel_size=(16, 16), stride=(8, 8))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (transformers): ModuleList(
        (0): FDViTLayer(
          (layers): ModuleList()
          (blocks): ModuleList(
            (0): Block(
              (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
              (attn): FDViTAttention(
                (qkv): Linear(in_features=64, out_features=192, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=64, out_features=64, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (get_v): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((64,), eps=1e-06, elementwise_affine=Tr

In [ ]:
# Extract and print the state dictionary (weights)
state_dict = model.state_dict()
print("Model Layer Names and Shapes:")
for layer_name, tensor in state_dict.items():
    print(f"{layer_name}: {tensor.shape}")

Model Layer Names and Shapes:
fdvit.encoder.pos_embed: torch.Size([1, 64, 27, 27])
fdvit.encoder.patch_embed.conv.weight: torch.Size([64, 3, 16, 16])
fdvit.encoder.patch_embed.conv.bias: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.norm1.weight: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.norm1.bias: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.attn.qkv.weight: torch.Size([192, 64])
fdvit.encoder.transformers.0.blocks.0.attn.qkv.bias: torch.Size([192])
fdvit.encoder.transformers.0.blocks.0.attn.proj.weight: torch.Size([64, 64])
fdvit.encoder.transformers.0.blocks.0.attn.proj.bias: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.attn.get_v.weight: torch.Size([64, 1, 3, 3])
fdvit.encoder.transformers.0.blocks.0.attn.get_v.bias: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.norm2.weight: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.norm2.bias: torch.Size([64])
fdvit.encoder.transformers.0.blocks.0.mlp.fc1.weight: torch.Size([256, 64

In [ ]:
# Initialize a variable to accumulate total size
total_size = 0

# Save the weights and their sizes to a text file
with open("FDViT_ti_weights_with_size.txt", "w") as file:
    for layer_name, weights in state_dict.items():
        # Calculate the size of the weights in bytes
        size_in_bytes = weights.element_size() * weights.numel()
        total_size += size_in_bytes

        # Write layer details
        file.write(f"Layer: {layer_name}\n")
        file.write(f"Shape: {tuple(weights.shape)}\n")
        file.write(f"Size (bytes): {size_in_bytes}\n")
        file.write(f"Weights:\n{weights.numpy()}\n\n")

# Print the total size of all weights
print(f"Total size of model weights: {total_size / (1024**2):.2f} MB")

Total size of model weights: 17.56 MB


Below is code for converting all the weights to int8

In [ ]:
import torch
from transformers import AutoModelForImageClassification

# Load the model
model = AutoModelForImageClassification.from_pretrained("amd/FDViT_ti", trust_remote_code=True)
model.eval()

# Extract the state dictionary (weights)
state_dict = model.state_dict()

# Define the quantization function
def quantize_weights(weights, bit_width=8):
    # Determine the range of the weights
    min_val = weights.min().item()
    max_val = weights.max().item()

    # Define quantization parameters
    quant_min = 0
    quant_max = 2**bit_width - 1
    scale = (max_val - min_val) / (quant_max - quant_min)
    zero_point = round(-min_val / scale)
    zero_point = max(quant_min, min(quant_max, zero_point))  # Clamp zero_point

    # Quantize the weights
    weights_quantized = torch.round(weights / scale + zero_point).clamp(quant_min, quant_max).to(torch.uint8)

    return weights_quantized, scale, zero_point

# Create a new state dictionary with quantized weights
quantized_state_dict = {}
quant_params = {}  # Store scale and zero point for each layer

total_original_size = 0
total_quantized_size = 0

for layer_name, weights in state_dict.items():
    # Compute the size of the original weights
    original_size = weights.element_size() * weights.numel()
    total_original_size += original_size

    if weights.dtype in [torch.float32, torch.float64]:  # Quantize only floating-point weights
        quantized_weights, scale, zero_point = quantize_weights(weights)
        quantized_state_dict[layer_name] = quantized_weights
        quant_params[layer_name] = {'scale': scale, 'zero_point': zero_point}

        # Compute the size of the quantized weights
        quantized_size = quantized_weights.element_size() * quantized_weights.numel()
        total_quantized_size += quantized_size
    else:
        # Copy non-floating-point weights as-is (e.g., biases)
        quantized_state_dict[layer_name] = weights
        total_quantized_size += weights.element_size() * weights.numel()  # Keep original size for biases

# Save the quantized weights
torch.save(quantized_state_dict, "FDViT_ti_quantized_weights.pth")

# Print total sizes
print("Total Model Weight Sizes:")
print(f"  Original Size: {total_original_size / (1024**2):.2f} MB")
print(f"  Quantized Size: {total_quantized_size / (1024**2):.2f} MB")
print(f"  Size Reduction: {(1 - total_quantized_size / total_original_size) * 100:.2f}%")

Total Model Weight Sizes:
  Original Size: 17.56 MB
  Quantized Size: 4.39 MB
  Size Reduction: 75.00%


In [ ]:
# Print an example of quantized weights and their parameters
for layer_name, quantized_weights in quantized_state_dict.items():
    if layer_name in quant_params:  # Only print for quantized layers
        print(f"Layer: {layer_name}")
        print(f"Quantized Weights: {quantized_weights}")
        print(f"Scale: {quant_params[layer_name]['scale']}, Zero Point: {quant_params[layer_name]['zero_point']}")
        break  # Print one layer as an example

In [ ]:
import numpy as np

In [ ]:
# Save quantized weights and their sizes to a text file

# Disable truncation for large arrays
np.set_printoptions(threshold=np.inf)

with open("FDViT_ti_quantized_weights_with_size.txt", "w") as file:
    total_quantized_size = 0
    for layer_name, quantized_weights in quantized_state_dict.items():
        # Calculate the size of the quantized weights in bytes
        quantized_size = quantized_weights.element_size() * quantized_weights.numel()
        total_quantized_size += quantized_size

        # Write layer information
        file.write(f"Layer: {layer_name}\n")
        file.write(f"Shape: {tuple(quantized_weights.shape)}\n")
        file.write(f"Quantized Size (bytes): {quantized_size}\n")

        # Check if the layer is quantized and write quantization parameters
        if layer_name in quant_params:
            scale = quant_params[layer_name]['scale']
            zero_point = quant_params[layer_name]['zero_point']
            file.write(f"Scale: {scale}\n")
            file.write(f"Zero Point: {zero_point}\n")

        # Write the actual quantized weights
        file.write(f"Quantized Weights:\n{quantized_weights.numpy()}\n\n")

    # Write total size of quantized weights
    file.write(f"Total Quantized Model Size: {total_quantized_size / (1024**2):.2f} MB\n")

print("Quantized weights and their sizes have been saved to 'FDViT_ti_quantized_weights_with_size.txt'.")


Quantized weights and their sizes have been saved to 'FDViT_ti_quantized_weights_with_size.txt'.


###Latency measurements of FD layer

In [ ]:
import torch
import time
from transformers import AutoModelForImageClassification

In [ ]:
# Access the first FD layer
fd_layer = model.fdvit.encoder.pools[0]

# Access components of the FD layer
conv_weight_shape = fd_layer.conv.weight.shape  # Convolution weights
ln_weight_shape = fd_layer.ln.weight.shape     # Layer normalization weights

# Print their shapes
print("First FD Layer Conv Weight Shape:", conv_weight_shape)
print("First FD Layer LN Weight Shape:", ln_weight_shape)

# Create a dummy tensor with the correct input shape
dummy_tensor = torch.randn(1, 27*27, 64)

# Move dummy tensor and model to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fd_layer = fd_layer.to(device)
dummy_tensor = dummy_tensor.to(device)

# Warmup for 10 cycles
for _ in range(10):
    _ = fd_layer(dummy_tensor)

# Measure latency over 10 cycles
num_cycles = 10
latencies = []

print("\nMeasuring latency over 10 cycles...")
for cycle in range(num_cycles):
    start_time = time.time()  # Start the timer
    _ = fd_layer(dummy_tensor)  # Forward pass through the FD layer
    latency = time.time() - start_time  # Calculate latency
    latencies.append(latency)  # Store the latency
    print(f"\tCycle {cycle + 1}: Latency = {latency:.6f} seconds")  # Display per-cycle latency

# Calculate the average latency
average_latency = sum(latencies) / num_cycles

# Print the results
print(f"Average Latency over {num_cycles} cycles: {average_latency:.6f} seconds")
print(f"\nOutput Shape: {dummy_tensor.shape}")  # Output shape remains unchanged

First FD Layer Conv Weight Shape: torch.Size([92, 64, 3, 3])
First FD Layer LN Weight Shape: torch.Size([64])

Measuring latency over 10 cycles...
	Cycle 1: Latency = 0.004285 seconds
	Cycle 2: Latency = 0.007405 seconds
	Cycle 3: Latency = 0.002500 seconds
	Cycle 4: Latency = 0.002036 seconds
	Cycle 5: Latency = 0.002273 seconds
	Cycle 6: Latency = 0.002183 seconds
	Cycle 7: Latency = 0.002290 seconds
	Cycle 8: Latency = 0.002232 seconds
	Cycle 9: Latency = 0.002148 seconds
	Cycle 10: Latency = 0.002857 seconds
Average Latency over 10 cycles: 0.003021 seconds

Output Shape: torch.Size([1, 729, 64])
